In [ ]:

import matplotlib
matplotlib.use("PDF")
import pyfits as pf
from pylab import *
import sys, glob
from scipy import stats

thedir='/sandbox/lsst/lsst/GUI/20161017_029-cte-timing/'
%cd $thedir

#%matplotlib inline

In [ ]:
#files = glob.glob('ITL-3800C-029_flat_light_004_20161010??????.fits')
files = sort(glob.glob('ITL-3800C-029_flat_light_031_20161017??????.fits'))
name = files[0][25:28]
print files
print name

In [ ]:
xstart = 505
xstop = 542
ov_start = 512
ystart = 200
ystop = 1800
xaxis = linspace(xstart,xstop-1,xstop-xstart)

for file in files:
    name = file[25:28]
    hdulist = pf.open(file, mode='readonly', do_not_scale_image_data=True)
    figure(figsize=(15,15))
    subplots_adjust(hspace = 0.5, wspace = 0.5)
    suptitle("Serial CTE by segment - ITL CCD 029 - %s"%name, fontsize = 16)

    for segment in range(16):
        try:
            subplot(4,4,segment+1)
            imhdr=hdulist[segment+1].header
            extname = imhdr['EXTNAME']
            title("CTE - %s"%extname)
            data = array(hdulist[segment+1].data + 32768, dtype = int32)
            flat_overscan = array(data[:,xstop-8:xstop].sum(axis = 1) / 8.0)
            cte_data = ((transpose(transpose(data) - flat_overscan))[ystart:ystop,:].mean(axis=0))[xstart:xstop]
            #print flat_overscan
            #print cte_data
            #sys.exit()
            cte_std = ((transpose(transpose(data) - flat_overscan))[ystart:ystop,:].std(axis=0) / sqrt(float(ystop-ystart)))[xstart:xstop]
            cti = (transpose(transpose(data) - flat_overscan))[ystart:ystop,ov_start].mean()\
            / (transpose(transpose(data) - flat_overscan))[ystart:ystop,ov_start-1].mean() / ov_start
            scatter(xaxis, cte_data, color = 'blue')
            errorbar(xaxis, cte_data, yerr=cte_std)
            text(xstart + 8, 20000, "CTI = %.2E"%cti, fontsize = 12)
            yscale('log',nonposy='clip')
            ylim(.01, 100000)
            xticks([510,520,530,540])
            xlabel("X (First overscan = 512)")
            ylabel("Signal(ADU)")
            print "Segment %s done"%extname
        except:
            continue
    savefig("Serial_CTE_%s_17Oct16.pdf"%name)
    clf()